In [1]:
import numpy as np
import pandas as pd
import glob as gb 
import os

### Состояние склада на каждый день

Создание имени файла

In [2]:
def create_filenames(letter, number):
    
    c = chr(ord('0') + number)
    str_i = 'MS-' + letter + c + '-inventory.csv'
    str_sl = 'MS-' + letter + c + '-sell.csv'
    str_spl = 'MS-' + letter + c + '-supply.csv'
    str_d = 'MS-' + letter + c + '-daily.csv'
    return str_i, str_sl, str_spl, str_d

Количество магазинов в штате

In [3]:
def num_shop(state):
    
    if state == 'b' or state == 'm':
        max = 2
    elif state == 's':
        max = 5
    return max

In [8]:
def daily_stor(state):
    
    max = num_shop(state)
    files = {}
    path = {}
    answer = {}
    shop = 0
    while shop < max:
        c = chr(ord('0') + shop + 1)
        path[shop] = gb.glob('./input/*'+ state + c + '*.csv')
        print(path[shop])
        files[shop] = {}
        for j in range(0, len(path[shop])):
            files[shop][j]= pd.read_csv(path[shop][j])
        shop = shop + 1
    INV = 0
    SELL = 1
    SPL = 2
    DAILY = 3
    
    shop = 0
    while shop < max:
        files[shop][SELL]['date'] = pd.to_datetime(files[shop][SELL]['date'], format='%Y-%m-%d')
        files[shop][INV]['date'] = pd.to_datetime(files[shop][INV]['date'], format='%Y-%m-%d')
        files[shop][SPL]['date'] = pd.to_datetime(files[shop][SPL]['date'], format='%Y-%m-%d')
        
        files[shop][SELL]['sku_num'] = files[shop][SELL]['sku_num'].apply(lambda x: 0 if x.find('ap') == 6 else 1)
        daily = pd.crosstab(files[shop][SELL]['date'], files[shop][SELL]['sku_num'], margins = True)
        daily = daily.drop(['All'])
        daily_new = pd.DataFrame(columns=['date', 'apple', 'pen'])
        daily_new['date'] = files[shop][SELL]['date'].drop_duplicates()
        daily_new['apple'] = daily[0].values
        daily_new['pen'] = daily[1].values
        daily_new.index = range(0, daily[0].size)
        print(daily_new.head())
        
        daily_new.index = files[shop][SELL]['date'].drop_duplicates()
        daily_new = daily_new.drop(['date'], axis=1)
        daily_new['apple'] = -daily_new['apple']
        daily_new['pen'] = -daily_new['pen']
        
        files[shop][SPL].index = files[shop][SPL].date
        files[shop][SPL] = files[shop][SPL].drop(['date'], axis=1)
        
        df = pd.concat([files[shop][SPL], daily_new]).sort_index()
        df = df.resample('D').sum()
        
        l = df.index.size
        df['apple_sum'] = df['apple'].rolling(str(l) + 'D').sum()
        df['pen_sum'] = df['pen'].rolling(str(l) + 'D').sum()
        df = df.drop(['apple', 'pen'], axis=1)
        df = df.rename(columns={'apple_sum': 'apple', 'pen_sum': 'pen'})
        print(df.head())
        
        
        name = create_filenames(state, shop + 1)[DAILY]
        df.to_csv('answer/' + name, index=False)
        answer[shop] = df
        shop = shop + 1
        
    return answer
    

In [9]:
def main():
    
    daily_b = daily_stor('b')
    print(daily_b)
    daily_m = daily_stor('m')
    print(daily_m)
    daily_s = daily_stor('s')
    print(daily_s)
    
main() 
    

['./input\\MS-b1-inventory.csv', './input\\MS-b1-sell.csv', './input\\MS-b1-supply.csv']
['./input\\MS-b2-inventory.csv', './input\\MS-b2-sell.csv', './input\\MS-b2-supply.csv']
        date  apple  pen
0 2006-01-01   1815  156
1 2006-01-02   1862  143
2 2006-01-03   1880  171
3 2006-01-04   1797  153
4 2006-01-05   1942  133
              apple     pen
date                       
2006-01-01  33271.0  2574.0
2006-01-02  31409.0  2431.0
2006-01-03  29529.0  2260.0
2006-01-04  27732.0  2107.0
2006-01-05  25790.0  1974.0
        date  apple  pen
0 2006-01-01   1815  156
1 2006-01-02   1862  143
2 2006-01-03   1880  171
3 2006-01-04   1797  153
4 2006-01-05   1942  133
              apple     pen
date                       
2006-01-01  33271.0  2574.0
2006-01-02  31409.0  2431.0
2006-01-03  29529.0  2260.0
2006-01-04  27732.0  2107.0
2006-01-05  25790.0  1974.0
{0:                 apple       pen
date                           
2006-01-01    33271.0    2574.0
2006-01-02    31409.0    2431.

['./input\\MS-s2-inventory.csv', './input\\MS-s2-sell.csv', './input\\MS-s2-supply.csv']
['./input\\MS-s3-inventory.csv', './input\\MS-s3-sell.csv', './input\\MS-s3-supply.csv']
['./input\\MS-s4-inventory.csv', './input\\MS-s4-sell.csv', './input\\MS-s4-supply.csv']
['./input\\MS-s5-inventory.csv', './input\\MS-s5-sell.csv', './input\\MS-s5-supply.csv']
        date  apple  pen
0 2006-01-01     96   11
1 2006-01-02     79    6
2 2006-01-03     83   15
3 2006-01-04     86   14
4 2006-01-05     97   12
             apple    pen
date                     
2006-01-01  3002.0  294.0
2006-01-02  2923.0  288.0
2006-01-03  2840.0  273.0
2006-01-04  2754.0  259.0
2006-01-05  2657.0  247.0
        date  apple  pen
0 2006-01-01     96   11
1 2006-01-02     79    6
2 2006-01-03     83   15
3 2006-01-04     86   14
4 2006-01-05     97   12
             apple    pen
date                     
2006-01-01  3002.0  294.0
2006-01-02  2923.0  288.0
2006-01-03  2840.0  273.0
2006-01-04  2754.0  259.0
2006-0